<a href="https://colab.research.google.com/github/MrPrabhathPeri/SMS-Text-Classification/blob/main/sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Load the dataset
train_dataset_path = "train-data.tsv"
test_dataset_path = "valid-data.tsv"

train_dataset = pd.read_csv(train_dataset_path, sep="\t", header=None)
test_dataset = pd.read_csv(test_dataset_path, sep="\t", header=None)

# Preprocess the data
train_messages = train_dataset[1].values
train_labels = train_dataset[0].map({"ham": 0, "spam": 1}).values

test_messages = test_dataset[1].values
test_labels = test_dataset[0].map({"ham": 0, "spam": 1}).values

# Tokenize and vectorize the messages
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    train_messages, target_vocab_size=2**13
)


train_messages_encoded = [tokenizer.encode(message) for message in train_messages]
test_messages_encoded = [tokenizer.encode(message) for message in test_messages]

# Pad sequences
max_length = max(len(message) for message in train_messages_encoded)
train_messages_padded = tf.keras.preprocessing.sequence.pad_sequences(train_messages_encoded, maxlen=max_length, padding="post")
test_messages_padded = tf.keras.preprocessing.sequence.pad_sequences(test_messages_encoded, maxlen=max_length, padding="post")

# Create the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

# Compile the model
model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])

# Train the model
history = model.fit(train_messages_padded, train_labels, epochs=10, validation_data=(test_messages_padded, test_labels))

# Function to predict messages based on the model
def predict_message(pred_text):
    pred_text_encoded = [tokenizer.encode(pred_text)]
    pred_text_padded = tf.keras.preprocessing.sequence.pad_sequences(pred_text_encoded, maxlen=max_length, padding="post")
    prediction = model.predict(pred_text_padded)[0][0]
    label = "spam" if prediction >= 0.5 else "ham"
    return [prediction, label]

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()